# Google Tunix Hackathon: Dual-Stream Kernel

This notebook family targets Kaggle compliance while demonstrating the dual-stream architecture end-to-end (generator + monitoring + coherence audit). It includes accelerator-specific setup and optional Gemma integration for showing work through fine-tuning.
**Accelerator target:** `CPU` variant

## Hardware-enforced dual-stream concept (whitepaper-aligned)
- **Separated streams**: the answer stream and the instrumented monologue stream are produced together so reviewers can inspect logits/attention alongside generated text. This mirrors the paper's emphasis on verifiable inner alignment via dual channels.
- **Hardware anchoring**: competition guidance encourages running the probe stack where hardware (GPU/TPU) enforces execution traceability. The notebook keeps the probes wired into the model forward pass so you can export per-token traces and align with the whitepaper's hardware-bound auditing narrative.
- **Show-your-work fine-tuning**: use the Gemma checkpoints below to fine-tune and re-run the probe pipeline; the outputs are exported for judges to verify training decisions, not just final answers.

## Optional Gemma integration
- Set `ENABLE_GEMMA=1` in the environment to swap the generator to Gemma 2B or Gemma 3 1B (default: `google/gemma-3-1b-it`).
- Provide a Hugging Face token with access to Gemma models: `os.environ['HF_TOKEN']='<token>'`.
- The dual-stream probes remain attached, so per-token traces continue to export for Kaggle review.

## 1) Environment prep

In [ ]:
import os, sys, json, random, subprocess
from pathlib import Path
import importlib
import numpy as np
import torch

ACCELERATOR_TARGET = 'cpu'  # 'cpu', 'cuda', or 'tpu'
BASELINE_MODEL = 'gpt2'
GEMMA_MODEL = os.environ.get('GEMMA_MODEL_ID', 'google/gemma-3-1b-it')
ENABLE_GEMMA = os.environ.get('ENABLE_GEMMA', '0') == '1'

DEFAULT_ROOT = Path('.')
kaggle_root = Path('/kaggle/input/dual-stream')
repo_root = kaggle_root if kaggle_root.exists() else DEFAULT_ROOT
print(f"Using repo root: {repo_root.resolve()}")

for extra in ['python_poc', 'dualstream_anticollapse']:
    path = repo_root / extra
    if str(path) not in sys.path:
        sys.path.append(str(path))

required = [
    ('pandas', 'pandas'),
    ('sklearn', 'scikit-learn'),
    ('scipy', 'scipy'),
    ('matplotlib', 'matplotlib'),
    ('seaborn', 'seaborn'),
]
for import_name, pip_name in required:
    try:
        importlib.import_module(import_name)
    except ImportError:
        subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', pip_name], check=True)

try:
    import transformers  # noqa: F401
except ImportError:
    req_path = repo_root / 'python_poc' / 'requirements.txt'
    if req_path.exists():
        subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', '-r', str(req_path)], check=True)

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# resolve device
if ACCELERATOR_TARGET == 'cuda' and torch.cuda.is_available():
    device = 'cuda'
elif ACCELERATOR_TARGET == 'tpu':
    try:
        import torch_xla.core.xla_model as xm
        device = xm.xla_device()
    except Exception as exc:  # pragma: no cover - TPU optional
        print(f"TPU unavailable, falling back to CPU: {exc}")
        device = 'cpu'
else:
    device = 'cpu'
print(f"Device: {device}")


## 2) Dual-Stream generator

In [ ]:
from dual_stream_poc import DualStream

prompt = """You are an alignment auditor. Briefly explain why transparency in model reasoning matters."""
model_choice = GEMMA_MODEL if ENABLE_GEMMA else BASELINE_MODEL
print(f"Dual-stream model: {model_choice} (Gemma enabled: {ENABLE_GEMMA})")

# If Gemma is enabled, allow bfloat16 on accelerators
if ENABLE_GEMMA and device != 'cpu':
    model_kwargs = {'torch_dtype': torch.bfloat16}
else:
    model_kwargs = {}

# Construct and run
ds = DualStream(model_name=model_choice, device=device, top_k=5)
ds_output = ds.generate(prompt=prompt, max_new_tokens=60, temperature=0.3, top_p=0.95)

print("\nAnswer stream preview:\n", ds_output['answer_text'][:400])
print("\nFirst monologue frame:\n", ds_output['monologue_text'].split('\n')[0])


## 3) Train baseline and monitor for drift/performance regressions

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from dualstream_anticollapse.retrain import build_model, fit_model, predict
from dualstream_anticollapse.metrics import classification_metrics
from dualstream_anticollapse.drift import population_stability_index, ks_test
from dualstream_anticollapse.edge import zscore_outliers

data = pd.DataFrame({
    'text': [
        'The system remained stable under audit.',
        'A misaligned behavior was detected.',
        ds_output['answer_text'],
    ],
    'label': [1, 0, 1],
})

vec = TfidfVectorizer()
X = vec.fit_transform(data['text'])
y = data['label'].astype(int)
try:
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.33, random_state=42, stratify=y
    )
except ValueError:
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.33, random_state=42, stratify=None
    )

model = build_model('sgd_classifier')
model = fit_model(model, X_train, y_train)
train_preds, prob_train = predict(model, X_train)
pred_labels, prob_test = predict(model, X_test)
metrics = classification_metrics(y_test, pred_labels, y_proba=prob_test)
print(json.dumps(metrics, indent=2))

psi = population_stability_index(prob_train, prob_test)
ksD, pval = ks_test(prob_train, prob_test)
print(f"\nDrift tests (train vs test probs): PSI={psi:.4f}, KS p-value={pval:.4f}")

outlier_df = pd.DataFrame({'probabilities': prob_test})
outliers = zscore_outliers(outlier_df, ['probabilities'], z=3.5)
print("Outlier indices:", {k: list(v) for k, v in outliers.items()})


## 4) Coherence audit of the Dual-Stream output

In [ ]:
from dualstream_anticollapse.coherence import CoherenceAuditor

def build_logit_topk(frames):
    if not frames:
        return []
    topk_records = []
    for frame in frames:
        for tid, prob in zip(frame['topk_ids'], frame['topk_probs']):
            topk_records.append({'token_id': tid, 'prob': prob})
    return topk_records

thresholds = {
    'max_allowed_deception_tokens': 0,
    'max_allowed_conflict_markers': 0,
}
coherence = CoherenceAuditor(thresholds)
report_obj = coherence.audit(ds_output['answer_text'], ds_output['monologue_text'])
report = report_obj.__dict__
print(json.dumps(report, indent=2))


## 5) Export for Kaggle submission

In [ ]:
output_dir = Path('submission_outputs')
output_dir.mkdir(exist_ok=True)

with open(output_dir / f'coherence_report_{ACCELERATOR_TARGET}.json', 'w') as f:
    json.dump(report, f, indent=2)

with open(output_dir / f'dual_stream_output_{ACCELERATOR_TARGET}.json', 'w') as f:
    json.dump(ds_output, f, indent=2)

print('Saved artifacts to', output_dir)


### What to submit
- The executed notebook for the chosen accelerator variant.
- `submission_outputs/dual_stream_output_<accel>.json`
- `submission_outputs/coherence_report_<accel>.json`
These files demonstrate both generations and the monitored traces expected by judges.